In [63]:
import unittest
from pydantic import BaseModel, Field, field_validator
import plotly.graph_objects as go
import numpy as np
from typing import List, Optional

uses a hexagonal coordinate system

ie, going clockwise around origin, pointy side up

| q | r | s |
 |----|--|--|
1 |-1 |0
1 |0 |-1
0 |1 |-1
-1| 1 |0
-1 |0 |1
0 |-1| 1

q is in the up right orientation  
r is rows in the down orientation  
s is in the up left direction  

two coordinates that are adjacant

In [64]:
class HexCoordinate(BaseModel):
    q: int
    r: int
    s: int

    @field_validator('s', mode='before')
    @classmethod
    def validate_cube_coordinates(cls, v, values):
        if values.data['q'] + values.data['r'] + v != 0:
            raise ValueError('Invalid cube coordinates')
        return v

class GamePiece(BaseModel):
    hex_coordinates: list[HexCoordinate]
    icon: str = "�"
    team: str

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

spider1 = GamePiece(hex_coordinates=[HexCoordinate(q=1, r=-1, s=0)], icon="🕷️", team='black')
grasshopper1 = GamePiece(hex_coordinates=[HexCoordinate(q=2, r=-2, s=0)], icon="🦗", team='white')

adjacant

In [65]:
def are_adjacent(coord1: GamePiece, coord2: GamePiece, verbose: bool) -> bool:
    dq = abs(coord1.q - coord2.q)
    dr = abs(coord1.r - coord2.r)
    ds = abs(coord1.s - coord2.s)
    distance = (dq + dr + ds) == 2
    if verbose:
        if distance:
            print("The coordinates are adjacent")
        else:
            print(f"The coordinates are not adjacent, distance is {dq + dr + ds}")

    return distance

are_adjacent(spider1.hex_coordinates[0], grasshopper1.hex_coordinates[0], True)

The coordinates are adjacent


True

In [66]:
def hex_to_pixel(coord: HexCoordinate, size: float = 1.0):
    """Convert hex coordinate to pixel position for plotting."""
    x = size * (3/2 * coord.q)
    y = size * (np.sqrt(3)/2 * coord.q + np.sqrt(3) * coord.r)
    return x, y

def get_hexagon_vertices(x: float, y: float, size: float = 1.0):
    """Get vertices of a hexagon centered at (x, y)."""
    angles = np.linspace(0, 2*np.pi, 7)  # 7 points to close the hexagon
    vertices_x = x + size * np.cos(angles)
    vertices_y = y + size * np.sin(angles)
    return vertices_x, vertices_y



In [ ]:
def visualize_game_board(pieces: list[GamePiece], show_empty_hexes: Optional[list[HexCoordinate]] = None):
    """
    Visualize game pieces on hex coordinates
    """
    fig = go.Figure()
    
    # CUSTOMIZATION: Adjust hexagon size here
    hex_size = 0.95
    
    # CUSTOMIZATION: Team colors mapping
    team_colors = {
        "black": "#1D1A1A",  # Light gray for black team
        "white": "#FFFFFF",  # White for white team
        # Add more team colors as needed
    }
    
    team_border_colors = {
        "black": "#000000",  # Black border
        "white": "#808080",  # Gray border for visibility
    }
    
    # Draw empty hexes if provided
    if show_empty_hexes:
        for coord in show_empty_hexes:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor='#F5F5F5',
                line=dict(color='lightgray', width=1),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'Empty: q={coord.q}, r={coord.r}, s={coord.s}',
            ))
    
    # Draw hexes with game pieces
    for piece in pieces:
        # Each piece can occupy multiple hexes
        for coord in piece.hex_coordinates:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            # Get team colors
            fill_color = team_colors.get(piece.team, 'lightgray')
            line_color = team_border_colors.get(piece.team, 'gray')
            
            # Draw the hexagon
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor=fill_color,
                line=dict(color=line_color, width=2),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'({piece.team})<br>Position: ({coord.q},{coord.r},{coord.s})',
            ))
            
            # Add piece icon in the center
            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y],
                mode='text',
                text=[piece.icon],
                textfont=dict(size=60, color='black'),  # CUSTOMIZATION: Icon size
                showlegend=False,
                hoverinfo='skip'
            ))
            
            # CUSTOMIZATION: Optionally show coordinate labels
            # Uncomment the lines below to show coordinates beneath icons
            # fig.add_trace(go.Scatter(
            #     x=[center_x],
            #     y=[center_y - 0.3],
            #     mode='text',
            #     text=[f'({coord.q},{coord.r},{coord.s})'],
            #     textfont=dict(size=8, color='gray'),
            #     showlegend=False,
            #     hoverinfo='skip'
            # ))
    
    # CUSTOMIZATION: Adjust plot appearance here
    fig.update_layout(
        title='Current State of the Game Board',
        xaxis=dict(
            scaleanchor='y',
            scaleratio=1,
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        plot_bgcolor='white',
        width=800,
        height=800,
    )
    
    fig.show()

visualize_game_board([spider1, grasshopper1], show_empty_hexes=[])